In [1]:
import pandas as pd
import os
import requests
import io
import pandas as pd

In [2]:
path = "/mnt/c/Users/Samuel/Documents/python/data/HESA"

In [3]:
def download_files(path):

    files = [
    'https://www.hesa.ac.uk/data-and-analysis/providers/estates/table-1.csv',
    'https://www.hesa.ac.uk/data-and-analysis/providers/business-community/table-2a.csv',
    'https://www.hesa.ac.uk/data-and-analysis/providers/business-community/table-2b.csv',
    'https://www.hesa.ac.uk/data-and-analysis/providers/business-community/table-1.csv',
    'https://www.hesa.ac.uk/data-and-analysis/providers/business-community/table-3.csv',
    'https://www.hesa.ac.uk/data-and-analysis/providers/business-community/table-5.csv']

    for f in files:
        download = requests.get(f)
        with open(os.path.join(path,f.split("/")[-2],f.split("/")[-1]), 'w') as temp_file:
            temp_file.writelines(download.content.decode('utf-8'))

In [36]:
def read_files(path):
    file_metadata = []
    file_data = {}
    for i, filename in enumerate([os.path.join(a[0], b) for a in os.walk(path) for b in a[2]]):
        with open(filename,'r') as f:
            x = f.readlines()
        blank_lines = [0==len(l.replace(",","").replace("\n","")) for l in x]
        idx_metadata_end = blank_lines.index(True)
        header = idx_metadata_end + blank_lines[idx_metadata_end:].index(False)
        meta = pd.read_csv(filename, nrows = idx_metadata_end, header=None, index_col=0).iloc[:,0].to_frame(i)
        meta.index.name=None
        df = pd.read_csv(filename, header=header)
        file_data[i]=df
        file_metadata.append(meta)
    file_metadata = pd.concat(file_metadata, axis=1).T
    return file_data, file_metadata

In [44]:
files, meta = read_files(path)

In [45]:
pd.set_option("display.max_colwidth",-1)
meta["Title"].to_frame()

,Title
0,"Income from collaborative research involving public funding by academic year, type of income, source of public funding and HE provider"
1,"Business and community services by type of service, academic year, type of organisation and HE provider"
2,Continuing Professional Development (CPD) and Continuing Education (CE) courses for business and the community by academic year and HE provider
3,"Income from regeneration and development programmes by academic year, programme and HE provider"
4,"Social, community and cultural engagement: designated public events by academic year, type of event, nature of event, number of attendees, academic staff time (days) and HE provider"
5,Estates management by academic year and HE provider


In [59]:
pd.set_option("display.max_rows",500)
categories = pd.Series(files[5]["Category marker"].unique()).to_frame("Categories")
providers = pd.Series(files[5]["HE Provider"].unique()).sort_values().to_frame("Providers")
pd.concat((categories,providers), axis=1).fillna("")

,Categories,Providers
0,Total income (£),Anglia Ruskin University
1,Teaching income (£),The Arts University Bournemouth
2,Research income (£),Bath Spa University
3,Other non-residential income (£),University College Birmingham
4,Non-residential income total (£),Bournemouth University
5,Total expenditure (£),The University of Brighton
6,Teaching student headcount,The University College of Osteopathy
7,Research student headcount,Brunel University London
8,Teaching student FTE,Buckinghamshire New University
9,Research student FTE,Canterbury Christ Church University


In [108]:
tmp = pd.Series(files[5]["Category marker"].unique()).to_frame("category")
tmp["units"] = tmp["category"].str.split("(").apply(lambda x : x[-1][:-1] if len (x)>1 else "")
tmp["category"] = tmp["category"].str.split("(").apply(lambda x : "(".join(x[:-1]) if len(x)>1 else x[0])
lcat = tmp["category"].str.lower().str

tmp["clss_carbon"] = lcat.contains("carbon emissions")
tmp["clss_renewable"] = lcat.contains("renewable energy")
tmp["clss_water"] = lcat.contains("water consumption") | lcat.contains("wastewater")
tmp["clss_waste"] = lcat.contains("waste mass")
tmp["clss_income"] = lcat.contains("income")
tmp["clss_expenditure"] = lcat.contains("expenditure")
tmp["clss_headcount"] = lcat.contains("headcount") | lcat.contains("fte")
tmp["clss_commuting"] = lcat.contains("commuting")
tmp["clss_energy"] = lcat.contains("energy consumption")
tmp["clss_fuel"] = lcat.contains("fuel used")
tmp["clss_gia"] = lcat.contains("gia")
tmp["clss_parking"] = lcat.contains("parking") | lcat.contains("cycle spaces")
tmp["clss_other"] = ~tmp[[x for x in tmp.columns if "clss_" in x]].any(axis=1)
tmp[tmp["clss_other"]]

tmp[tmp[[x for x in tmp.columns if "clss_" in x]].sum(axis=1)>1]

,category,units,clss_carbon,clss_renewable,clss_water,clss_waste,clss_income,clss_expenditure,clss_headcount,clss_commuting,clss_energy,clss_fuel,clss_gia,clss_parking,clss_other
102,Scope 3 carbon emissions from staff commuting by air,tonnes CO2e,True,False,False,False,False,False,False,True,False,False,False,False,False
103,Scope 3 carbon emissions from staff commuting by rail,tonnes CO2e,True,False,False,False,False,False,False,True,False,False,False,False,False
104,Scope 3 carbon emissions from staff commuting by trams,tonnes CO2e,True,False,False,False,False,False,False,True,False,False,False,False,False
105,Scope 3 carbon emissions from staff commuting by underground,tonnes CO2e,True,False,False,False,False,False,False,True,False,False,False,False,False
106,Scope 3 carbon emissions from staff commuting by public bus,tonnes CO2e,True,False,False,False,False,False,False,True,False,False,False,False,False
107,Scope 3 carbon emissions from staff commuting by coach,tonnes CO2e,True,False,False,False,False,False,False,True,False,False,False,False,False
108,Scope 3 carbon emissions from staff commuting by car,tonnes CO2e,True,False,False,False,False,False,False,True,False,False,False,False,False
109,Scope 3 carbon emissions from staff commuting by taxi,tonnes CO2e,True,False,False,False,False,False,False,True,False,False,False,False,False
110,Scope 3 carbon emissions from staff commuting by motorcycle or moped,tonnes CO2e,True,False,False,False,False,False,False,True,False,False,False,False,False
111,Scope 3 carbon emissions from staff commuting by ferry,tonnes CO2e,True,False,False,False,False,False,False,True,False,False,False,False,False
